# LLMs - 大语言模型接口详解

## 概述

LLMs（Large Language Models）是 LangChain 中用于与传统文本生成模型交互的接口。与 Chat Models 不同，LLMs 接受字符串作为输入并返回字符串，更适合文本补全任务。

### LLMs vs Chat Models 对比

| 特性 | LLMs | Chat Models |
|------|------|-------------|
| 输入格式 | 字符串 | 消息列表 |
| 输出格式 | 字符串 | 消息对象 |
| 适用场景 | 文本补全、生成 | 对话、聊天 |
| 接口方法 | `invoke()`, `generate()` | `invoke()`, `stream()` |

### 核心特性

- **文本补全**: 基于提示词生成文本
- **批量生成**: 支持批量文本生成
- **流式输出**: 支持实时流式生成
- **参数控制**: 温度、最大长度等参数
- **缓存支持**: 内置缓存机制
- **回调系统**: 支持生成过程监控

## 完整代码示例

### 引入依赖

In [1]:
"""
LangChain 0.3 LLMs 完整示例
"""
# 引入依赖
# from langchain_community.llms import Ollama
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.callbacks import BaseCallbackHandler
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
import asyncio
import time
from typing import List, Dict, Any

# 设置缓存
set_llm_cache(InMemoryCache())

### 1. 基础 LLM 使用


#### `invoke()` vs `generate()`
在 `LangChain` 中使用 `OllamaLLM`（或更广义地，所有 LLM 接口如 `ChatOpenAI`、`ChatOllama`）时，常会看到两个方法：

---

✅ 1. `invoke()`

**用途：执行一个完整的调用链，通常配合 LCEL 使用。**

* `invoke()` 是 LangChain `Runnable` 接口的方法，用于**单次输入、单次输出**。
* 输入是一个**完整结构**（如包含 prompt 字段或 messages），LangChain 会负责组织成模型能理解的格式。

🔧 示例：

```python
from langchain.chat_models import ChatOllama

llm = ChatOllama(model="llama3")

response = llm.invoke("你好，介绍一下你自己")
print(response.content)
```

📌 等价于说：“我给你一句话，你回复一句话。” —— 通常**返回的是 Message 对象**。

---

 ✅ 2. `generate()`

**用途：批量生成，适用于多个输入生成多个输出。**

* `generate()` 更底层，接受多个输入（列表），返回结构化的结果（含 token 使用量、messages 等）。
* 返回的是 `LLMResult` 对象，适合做**批量推理、统计分析、缓存优化**等。

🔧 示例：

```python
from langchain.chat_models import ChatOllama

llm = ChatOllama(model="llama3")

results = llm.generate([
    ["你好，介绍一下你自己"],
    ["讲一个笑话"]
])

for generation in results.generations:
    print(generation[0].text)
```

📌 一次生成多个回复，并返回详细信息（如 `generation_info`、token数等）。

---

✨ 总结对比

| 对比项  | `invoke()`              | `generate()`                  |
| ---- | ----------------------- | ----------------------------- |
| 输入格式 | 单条文本或结构化输入（字典、message）  | 列表（多个输入）                      |
| 返回类型 | 通常是 `AIMessage` / `str` | `LLMResult`（包含多个 generations） |
| 使用场景 | 单次调用（配合 LCEL、Runnable）  | 批量生成（如评估任务、缓存）                |
| 易用性  | ✅ 更适合简单使用               | 🛠️ 更适合底层控制、多样化需求             |

---



#### basic_llm_usage

In [8]:

# 1. 基础 LLM 使用
def basic_llm_usage():
    """基础 LLM 使用示例"""
    print("=" * 50)
    print("1. 基础 LLM 使用")
    print("=" * 50)

    # 初始化 Ollama LLM
    llm = OllamaLLM(
        base_url="http://localhost:11434",
        model="qwen2.5:3b",
        temperature=0.7
    )

    # 基础文本生成
    prompt = "请写一个关于人工智能的简短介绍："
    response = llm.invoke(prompt)
    print(f"生成文本: {response}")

    # 使用 generate 方法
    prompts = [
        "Python是什么？",
        "机器学习的定义是什么？",
        "深度学习有哪些应用？"
    ]

    generations = llm.generate(prompts)
    print(f"\n批量生成结果:")
    for i, gen in enumerate(generations.generations):
        print(f"问题{i+1}: {gen[0].text[:100]}...")
basic_llm_usage()

1. 基础 LLM 使用
生成文本: 人工智能（Artificial Intelligence，简称AI）是指通过模拟、扩展和增强人类智能的技术。它利用计算技术实现智能机器系统，使机器能够在无需人工干预的情况下执行复杂任务，并具备学习能力。

在人工智能领域中，机器学习是一种核心方法，允许计算机从数据中自动学习并提取规律或特征，而不需要显式编程。深度学习是机器学习的一个分支，它使用多层神经网络来模仿人脑如何处理信息，从而实现对复杂数据的高效理解和预测。

除此之外，自然语言处理使得计算机能够理解、解释和生成人类语言；计算机视觉技术让机器能够“看”并从中提取有用的信息；强化学习则使机器能够在环境中通过与环境交互以获得奖励或惩罚的方式进行学习。

随着算法和技术的进步，人工智能已经在医疗诊断、自动驾驶汽车、语音识别以及复杂的决策支持系统等多个领域取得了显著的应用成果。此外，人工智能还在不断地挑战和拓展人类的认知边界，并且为社会带来了前所未有的变革与发展机遇。

批量生成结果:
问题1: Python是一种高级编程语言，由Guido van Rossum于1980年代末开始设计和开发。它最初被称为“Active”但很快改名为“Python”，取自英国喜剧小品演员Eric Idle的名字...
问题2: 机器学习是一种人工智能（AI）的技术，使计算机能够在不进行明确编程的情况下通过数据学习和改进。简单来说，就是让计算机从历史数据中自动识别模式、提取特征，并利用这些信息来做出预测或决策。

在实际应用中...
问题3: 深度学习是一种机器学习的方法，它已经在多个领域都有了广泛的应用。以下是一些主要的应用场景：

1. 语音识别：深度学习在这一领域的应用已经非常成熟和成功，无论是基于手机的数字助理、智能音箱还是专业的语...


### 2. 流式生成示例

In [3]:

# 2. 流式生成示例
def streaming_generation():
    """流式生成示例"""
    print("\n" + "=" * 50)
    print("2. 流式生成示例")
    print("=" * 50)

    llm = OllamaLLM(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    prompt = "请详细解释什么是区块链技术："

    print("流式输出:")
    for chunk in llm.stream(prompt):
        print(chunk, end="", flush=True)
    print("\n")
streaming_generation()


2. 流式生成示例
流式输出:
区块链是一种去中心化的数据记录方式，它通过密码学方法确保信息的安全性和完整性，并且所有的交易信息都会被分布式地保存在一个公开的数据库中。以下是对区块链技术的详细解释：

1. 去中心化架构：
传统的数据库系统大多由单个组织或机构控制，因此存在风险：如单一节点失败、人为错误和数据篡改等。而区块链则采用去中心化的形式，每个参与者都是网络的一部分，无需依赖任何中央权威机构。

2. 分布式记账技术：
在区块链中，所有的交易记录都被分布在多个节点上进行保存，并通过密码学手段保证记录的安全性和完整性。这意味着即使有一个或几个节点出现故障或者被破坏，也不会对整个系统的正常运行造成影响。

3. 加密算法和共识机制：
每笔交易都会被打包成区块并在整个网络中进行广播以供其他节点验证。一旦多个节点都确认了交易的有效性，就会创建一个新的区块并将其添加到区块链的末端。这种技术确保了数据的安全性和防篡改特性。

4. 匿名性和透明度：
虽然每个用户都有一个独特的公钥身份用于与其他用户的交互和管理资金，但用户的真实姓名和个人信息通常不会直接公开在公共链上。同时，由于每笔交易都会被记录下来并可供所有参与者查看，因此可以保证整个系统的透明性。

5. 智能合约：
智能合约是指自动执行的代码，在满足特定条件时会触发预设的操作或行动。它们通常部署在区块链网络之上，并能够与现有的金融服务集成使用。

6. 跨链通信：
目前大多数区块链系统是独立运行并互不兼容的，这意味着不同网络之间的数据交换存在挑战。为了解决这个问题，跨链技术允许在不同的区块链之间传输信息和价值。

通过上述特点，区块链技术已被广泛应用于数字货币、供应链管理以及智能合约等领域，并展现出巨大的潜力和发展前景。



### 3. 使用提示模板

In [ ]:
# 3. 使用提示模板
def with_prompt_templates():
    """使用提示模板"""
    print("\n" + "=" * 50)
    print("3. 使用提示模板")
    print("=" * 50)

    llm = OllamaLLM(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 创建提示模板
    template = """
    作为一个{role}，请用{style}的风格回答以下问题：

    问题：{question}

    回答：
    """

    prompt = PromptTemplate(
        input_variables=["role", "style", "question"],
        template=template
    )

    # 创建链
    chain = prompt | llm | StrOutputParser()

    # 测试不同角色和风格
    examples = [
        {"role": "历史学家", "style": "学术", "question": "秦始皇统一中国的意义"},
        {"role": "诗人", "style": "浪漫", "question": "春天的特点"},
        {"role": "程序员", "style": "技术", "question": "什么是算法复杂度"}
    ]

    for example in examples:
        response = chain.invoke(example)
        print(f"\n{example['role']}({example['style']})回答:")
        print(response[:200] + "...")

### 4. 批量处理优化

In [ ]:

# 4. 批量处理优化
def batch_processing():
    """批量处理示例"""
    print("\n" + "=" * 50)
    print("4. 批量处理示例")
    print("=" * 50)

    llm = OllamaLLM(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 批量提示
    prompts = [
        "解释什么是机器学习",
        "Python的优势是什么",
        "什么是云计算",
        "区块链的应用场景",
        "人工智能的发展历程"
    ]

    # 计时批量处理
    start_time = time.time()
    responses = llm.batch(prompts)
    batch_time = time.time() - start_time

    print(f"批量处理{len(prompts)}个请求耗时: {batch_time:.2f}秒")

    for i, response in enumerate(responses):
        print(f"\n问题{i+1}: {prompts[i]}")
        print(f"回答: {response[:100]}...")

### 5. 参数调优示例

In [ ]:

# 5. 参数调优示例
def parameter_tuning():
    """参数调优示例"""
    print("\n" + "=" * 50)
    print("5. 参数调优示例")
    print("=" * 50)

    prompt = "创作一个科幻故事的开头："

    # 不同温度设置
    temperatures = [0.1, 0.5, 0.9, 1.2]

    for temp in temperatures:
        llm = Ollama(
            base_url="http://localhost:11434",
            model="qwen2.5:3b",
            temperature=temp
        )

        response = llm.invoke(prompt)
        print(f"\n温度 {temp}:")
        print(response[:150] + "...")

### 6. 自定义回调处理

In [5]:
# 6. 自定义回调处理
class CustomCallbackHandler(BaseCallbackHandler):
    """自定义回调处理器"""

    def __init__(self):
        self.tokens = []
        self.start_time = None

    def on_llm_start(self, serialized, prompts, **kwargs):
        self.start_time = time.time()
        print(f"开始生成，提示数量: {len(prompts)}")

    def on_llm_new_token(self, token: str, **kwargs):
        self.tokens.append(token)
        print(f"新token: '{token}'", end="")

    def on_llm_end(self, response, **kwargs):
        duration = time.time() - self.start_time
        total_tokens = len(self.tokens)
        print(f"\n生成完成，耗时: {duration:.2f}秒，token数: {total_tokens}")
        self.tokens = []

def callback_example():
    """回调示例"""
    print("\n" + "=" * 50)
    print("6. 回调示例")
    print("=" * 50)

    llm = OllamaLLM(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    callback_handler = CustomCallbackHandler()

    response = llm.invoke(
        "请简单介绍一下量子计算",
        config={"callbacks": [callback_handler]}
    )
callback_example()


6. 回调示例
开始生成，提示数量: 1
新token: '量子'新token: '计算'新token: '是一种'新token: '基于'新token: '量子'新token: '力学'新token: '原理'新token: '的'新token: '计算'新token: '方式'新token: '，'新token: '它'新token: '利用'新token: '量子'新token: '位'新token: '（'新token: '量子'新token: '比特'新token: '或'新token: 'q'新token: 'ubit'新token: '）'新token: '来'新token: '存储'新token: '和'新token: '处理'新token: '信息'新token: '。'新token: '与'新token: '经典'新token: '计算机'新token: '使用的'新token: '二'新token: '进'新token: '制'新token: '位'新token: '（'新token: '0'新token: ' 或'新token: ' '新token: '1'新token: '）'新token: '不同'新token: '的是'新token: '，'新token: '量子'新token: '位'新token: '可以'新token: '同时'新token: '表示'新token: '0'新token: ' 和'新token: ' '新token: '1'新token: ' 的'新token: '叠加'新token: '态'新token: '，'新token: '这'新token: '被称为'新token: '“'新token: '量子'新token: '叠加'新token: '”。

'新token: '此外'新token: '，'新token: '量子'新token: '比特'新token: '还可以'新token: '实现'新token: '一种'新token: '称为'新token: '“'新token: '量子'新token: '纠缠'新token: '”的'新token: '现象'新token: '，在'新token: '这种'新token: '状态下'新token: '，'新token: '一个'新token: '量子'新toke

### 7. 缓存机制示例

In [6]:

# 7. 缓存机制示例
def caching_example():
    """缓存机制示例"""
    print("\n" + "=" * 50)
    print("7. 缓存机制示例")
    print("=" * 50)

    llm = OllamaLLM(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    prompt = "什么是深度学习？"

    # 第一次调用（无缓存）
    start_time = time.time()
    response1 = llm.invoke(prompt)
    first_call_time = time.time() - start_time

    # 第二次调用（有缓存）
    start_time = time.time()
    response2 = llm.invoke(prompt)
    second_call_time = time.time() - start_time

    print(f"第一次调用耗时: {first_call_time:.2f}秒")
    print(f"第二次调用耗时: {second_call_time:.2f}秒")
    print(f"缓存加速比: {first_call_time/second_call_time:.2f}x")
    print(f"响应一致性: {response1 == response2}")
caching_example()


7. 缓存机制示例
第一次调用耗时: 8.98秒
第二次调用耗时: 0.00秒
缓存加速比: 36483.76x
响应一致性: True


### 8. 异步处理示例

In [7]:
# 8. 异步处理示例
async def async_llm_example():
    """异步处理示例"""
    print("\n" + "=" * 50)
    print("8. 异步处理示例")
    print("=" * 50)

    llm = OllamaLLM(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 异步单次调用
    response = await llm.ainvoke("异步调用测试")
    print(f"异步响应: {response[:100]}...")

    # 异步批量调用
    prompts = [
        "什么是API？",
        "什么是数据库？",
        "什么是网络协议？"
    ]

    start_time = time.time()
    responses = await llm.abatch(prompts)
    async_time = time.time() - start_time

    print(f"\n异步批量处理耗时: {async_time:.2f}秒")
    for i, response in enumerate(responses):
        print(f"问题{i+1}回答: {response[:80]}...")

    # 异步流式处理
    print("\n异步流式输出:")
    async for chunk in llm.astream("请介绍一下云原生技术"):
        print(chunk, end="", flush=True)
    print("\n")

# Jupyter Notebook 中的正确运行方式
print("\n运行异步示例...")
await async_llm_example()


运行异步示例...

8. 异步处理示例
异步响应: 异步调用是一种在不阻塞主线程的情况下进行请求或执行操作的技术。它允许一个操作开始后，即使客户端还在等待其他任务完成时，另一个任务已经开始处理。这通常用于提高用户体验和系统的响应性。下面我将通过几个示例...


CancelledError: 

### 9. 复杂链式处理

In [6]:

# 9. 复杂链式处理
def complex_chain_example():
    """复杂链式处理示例"""
    print("\n" + "=" * 50)
    print("9. 复杂链式处理示例")
    print("=" * 50)

    llm = OllamaLLM(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 创建多步骤处理链
    step1_template = PromptTemplate(
        input_variables=["topic"],
        template="请列出关于{topic}的5个关键点："
    )

    step2_template = PromptTemplate(
        input_variables=["key_points"],
        template="基于以下关键点，写一个简短的总结：\n{key_points}\n\n总结："
    )

    # 构建链
    def process_topic(topic: str) -> str:
        # 第一步：生成关键点
        key_points = (step1_template | llm).invoke({"topic": topic})

        # 第二步：生成总结
        summary = (step2_template | llm).invoke({"key_points": key_points})

        return {
            "topic": topic,
            "key_points": key_points,
            "summary": summary
        }

    # 测试
    result = process_topic("人工智能")
    print(f"主题: {result['topic']}")
    print(f"关键点: {result['key_points'][:200]}...")
    print(f"总结: {result['summary'][:200]}...")
complex_chain_example()


9. 复杂链式处理示例
主题: 人工智能
关键点: 当然，关于人工智能（AI）的五个关键点如下：

1. **智能自动化**：人工智能技术让机器能够模拟人类的认知过程，通过学习、推理和自我修正来完成复杂的任务。这不仅包括语音识别、图像处理等感知功能，还涉及自然语言理解与生成。

2. **数据驱动**：人工智能的发展高度依赖于大数据的支持，大量的数据用于训练模型并不断提高算法的准确性和效率。从社交媒体信息到医疗记录、电子商务交易等各种领域的数据都能...
总结: 以下是一个基于您提供的五个关键点的简短总结：

人工智能（AI）正通过智能自动化、数据驱动和机器学习等核心技术改变我们的生活和工作方式。它依赖于大数据的支持，并在伦理与隐私问题以及对社会影响方面带来了一系列挑战。AI不仅重塑了各个行业的运作模式，还推动教育体系变革以培养适应新时代的技能人才。此外，跨学科合作是推动人工智能发展的关键因素。

这些核心要点有助于我们全面理解AI技术及其未来的发展趋势和...


### 10. 并行处理示例

In [ ]:

# 10. 并行处理示例
def parallel_processing():
    """并行处理示例"""
    print("\n" + "=" * 50)
    print("10. 并行处理示例")
    print("=" * 50)

    llm = Ollama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 创建并行处理链
    parallel_chain = RunnableParallel({
        "definition": PromptTemplate.from_template("定义：{concept}是什么？") | llm,
        "advantages": PromptTemplate.from_template("优势：{concept}有什么优势？") | llm,
        "applications": PromptTemplate.from_template("应用：{concept}有哪些应用场景？") | llm,
        "challenges": PromptTemplate.from_template("挑战：{concept}面临哪些挑战？") | llm
    })

    # 执行并行处理
    concept = "区块链技术"
    start_time = time.time()
    results = parallel_chain.invoke({"concept": concept})
    parallel_time = time.time() - start_time

    print(f"并行处理耗时: {parallel_time:.2f}秒")
    print(f"\n概念: {concept}")

    for aspect, content in results.items():
        print(f"\n{aspect}: {content[:100]}...")

### 11. 错误处理和重试

In [ ]:

# 11. 错误处理和重试
def error_handling_example():
    """错误处理示例"""
    print("\n" + "=" * 50)
    print("11. 错误处理示例")
    print("=" * 50)

    # 正常模型
    llm = Ollama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 错误模型（不存在）
    error_llm = Ollama(
        base_url="http://localhost:11434",
        model="nonexistent-model"
    )

    def safe_invoke(llm, prompt: str, max_retries: int = 3):
        """安全调用函数"""
        for attempt in range(max_retries):
            try:
                response = llm.invoke(prompt)
                return response
            except Exception as e:
                print(f"尝试 {attempt + 1} 失败: {e}")
                if attempt == max_retries - 1:
                    return f"所有尝试都失败了，最后错误: {e}"
                time.sleep(1)  # 等待重试

    # 测试正常调用
    response = safe_invoke(llm, "测试正常调用")
    print(f"正常调用结果: {response[:50]}...")

    # 测试错误处理
    error_response = safe_invoke(error_llm, "测试错误处理")
    print(f"错误处理结果: {error_response}")

### 12. 自定义输出格式化

In [ ]:

# 12. 自定义输出格式化
def custom_output_formatting():
    """自定义输出格式化"""
    print("\n" + "=" * 50)
    print("12. 自定义输出格式化")
    print("=" * 50)

    llm = Ollama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 自定义格式化器
    class JSONFormatter:
        def format(self, text: str, metadata: Dict[str, Any] = None) -> Dict[str, Any]:
            return {
                "content": text.strip(),
                "length": len(text),
                "word_count": len(text.split()),
                "metadata": metadata or {}
            }

    formatter = JSONFormatter()

    # 创建格式化链
    def formatted_invoke(prompt: str) -> Dict[str, Any]:
        response = llm.invoke(prompt)
        return formatter.format(response, {"prompt": prompt})

    # 测试
    result = formatted_invoke("什么是微服务架构？")
    print(f"格式化结果:")
    for key, value in result.items():
        if key == "content":
            print(f"{key}: {str(value)[:100]}...")
        else:
            print(f"{key}: {value}")

## 不同模型提供商示例

In [ ]:
## 不同模型提供商示例

"""
不同 LLM 提供商示例
"""
from langchain_community.llms import Ollama
from langchain_openai import OpenAI
from langchain_community.llms import Bedrock
from langchain_core.prompts import PromptTemplate

def ollama_example():
    """Ollama 本地模型示例"""
    print("=" * 30)
    print("Ollama 示例")
    print("=" * 30)

    llm = Ollama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b",
        temperature=0.7
    )

    response = llm.invoke("解释什么是容器化技术")
    print(f"Ollama 响应: {response[:100]}...")

def openai_example():
    """OpenAI 示例"""
    print("\n" + "=" * 30)
    print("OpenAI 示例")
    print("=" * 30)

    try:
        llm = OpenAI(
            model="gpt-3.5-turbo-instruct",
            temperature=0.7,
            max_tokens=100
        )

        response = llm.invoke("解释什么是容器化技术")
        print(f"OpenAI 响应: {response}")
    except Exception as e:
        print(f"OpenAI 调用失败: {e}")

def bedrock_example():
    """AWS Bedrock 示例"""
    print("\n" + "=" * 30)
    print("AWS Bedrock 示例")
    print("=" * 30)

    try:
        llm = Bedrock(
            model_id="anthropic.claude-v2",
            region_name="us-east-1"
        )

        # Claude 需要特定格式
        prompt = "Human: 解释什么是容器化技术\n\nAssistant:"
        response = llm.invoke(prompt)
        print(f"Bedrock 响应: {response[:100]}...")
    except Exception as e:
        print(f"Bedrock 调用失败: {e}")

def main():
    """运行所有提供商示例"""
    ollama_example()
    openai_example()
    bedrock_example()

if __name__ == "__main__":
    main()


## 高级用法示例

In [ ]:

"""
LLM 高级用法示例
"""
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.runnables import RunnableBranch, RunnableLambda
from typing import Dict, Any

### 少样本学习示例

In [ ]:

def few_shot_learning():
    """少样本学习示例"""
    print("=" * 50)
    print("少样本学习示例")
    print("=" * 50)

    llm = Ollama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 定义示例
    examples = [
        {
            "question": "什么是Python？",
            "answer": "Python是一种高级编程语言，以其简洁的语法和强大的功能而闻名。"
        },
        {
            "question": "什么是JavaScript？",
            "answer": "JavaScript是一种主要用于网页开发的编程语言，可以创建交互式网页。"
        },
        {
            "question": "什么是SQL？",
            "answer": "SQL是结构化查询语言，用于管理和操作关系数据库中的数据。"
        }
    ]

    # 创建示例模板
    example_template = """
    问题: {question}
    答案: {answer}
    """

    example_prompt = PromptTemplate(
        input_variables=["question", "answer"],
        template=example_template
    )

    # 创建少样本提示模板
    few_shot_prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        prefix="以下是一些问答示例：",
        suffix="问题: {input}\n答案:",
        input_variables=["input"]
    )

    # 创建链
    chain = few_shot_prompt | llm

    # 测试
    response = chain.invoke({"input": "什么是Java？"})
    print(f"少样本学习回答: {response}")

### 条件路由示例

In [ ]:

def conditional_routing():
    """条件路由示例"""
    print("\n" + "=" * 50)
    print("条件路由示例")
    print("=" * 50)

    llm = Ollama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 定义不同类型的提示模板
    tech_template = PromptTemplate(
        input_variables=["question"],
        template="作为技术专家，请详细回答：{question}"
    )

    general_template = PromptTemplate(
        input_variables=["question"],
        template="请简单回答：{question}"
    )

    creative_template = PromptTemplate(
        input_variables=["question"],
        template="请用创意的方式回答：{question}"
    )

    # 路由函数
    def route_question(input_dict: Dict[str, Any]) -> str:
        question = input_dict["question"].lower()
        if any(word in question for word in ["编程", "技术", "算法", "代码"]):
            return "tech"
        elif any(word in question for word in ["创意", "故事", "诗歌", "想象"]):
            return "creative"
        else:
            return "general"

    # 创建条件分支
    routing_chain = RunnableBranch(
        (lambda x: route_question(x) == "tech", tech_template | llm),
        (lambda x: route_question(x) == "creative", creative_template | llm),
        general_template | llm  # 默认分支
    )

    # 测试不同类型的问题
    questions = [
        "什么是机器学习算法？",
        "写一个关于月亮的诗",
        "今天天气怎么样？"
    ]

    for question in questions:
        route_type = route_question({"question": question})
        response = routing_chain.invoke({"question": question})
        print(f"\n问题类型: {route_type}")
        print(f"问题: {question}")
        print(f"回答: {response[:100]}...")

### 记忆模拟示例

In [ ]:

def memory_simulation():
    """记忆模拟示例"""
    print("\n" + "=" * 50)
    print("记忆模拟示例")
    print("=" * 50)

    llm = Ollama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 简单的记忆存储
    conversation_memory = []

    def chat_with_memory(user_input: str) -> str:
        # 构建包含历史的提示
        context = ""
        if conversation_memory:
            context = "对话历史：\n"
            for i, (q, a) in enumerate(conversation_memory[-3:]):  # 只保留最近3轮
                context += f"用户: {q}\nAI: {a}\n"
            context += "\n"

        prompt = f"{context}当前问题: {user_input}\n请回答:"

        # 生成回答
        response = llm.invoke(prompt)

        # 保存到记忆
        conversation_memory.append((user_input, response))

        return response

    # 模拟对话
    conversation = [
        "我是一名软件工程师",
        "我主要使用Python开发",
        "你知道我的职业是什么吗？",
        "我使用什么编程语言？"
    ]

    for user_msg in conversation:
        ai_response = chat_with_memory(user_msg)
        print(f"\n用户: {user_msg}")
        print(f"AI: {ai_response[:100]}...")

In [ ]:

def main():
    """运行所有高级示例"""
    few_shot_learning()
    conditional_routing()
    memory_simulation()

if __name__ == "__main__":
    main()



### 性能优化示例

In [ ]:

"""
LLM 性能优化示例
"""
from langchain_community.llms import Ollama
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.globals import set_llm_cache
from langchain_core.prompts import PromptTemplate
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor
from typing import List

#### 缓存性能对比

In [ ]:

def caching_comparison():
    """缓存性能对比"""
    print("=" * 50)
    print("缓存性能对比")
    print("=" * 50)

    llm = Ollama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    prompt = "什么是人工智能？"

    # 无缓存测试
    set_llm_cache(None)
    start_time = time.time()
    response1 = llm.invoke(prompt)
    no_cache_time = time.time() - start_time

    # 内存缓存测试
    set_llm_cache(InMemoryCache())
    start_time = time.time()
    response2 = llm.invoke(prompt)  # 第一次调用
    first_cache_time = time.time() - start_time

    start_time = time.time()
    response3 = llm.invoke(prompt)  # 第二次调用（命中缓存）
    second_cache_time = time.time() - start_time

    print(f"无缓存耗时: {no_cache_time:.2f}秒")
    print(f"首次缓存耗时: {first_cache_time:.2f}秒")
    print(f"命中缓存耗时: {second_cache_time:.2f}秒")
    print(f"缓存加速比: {no_cache_time/second_cache_time:.2f}x")

#### 批量处理 vs 顺序处理性能对比

In [ ]:

def batch_vs_sequential():
    """批量处理 vs 顺序处理性能对比"""
    print("\n" + "=" * 50)
    print("批量处理 vs 顺序处理性能对比")
    print("=" * 50)

    llm = Ollama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    prompts = [
        "什么是云计算？",
        "什么是大数据？",
        "什么是物联网？",
        "什么是5G技术？",
        "什么是边缘计算？"
    ]

    # 顺序处理
    start_time = time.time()
    sequential_responses = []
    for prompt in prompts:
        response = llm.invoke(prompt)
        sequential_responses.append(response)
    sequential_time = time.time() - start_time

    # 批量处理
    start_time = time.time()
    batch_responses = llm.batch(prompts)
    batch_time = time.time() - start_time

    print(f"顺序处理耗时: {sequential_time:.2f}秒")
    print(f"批量处理耗时: {batch_time:.2f}秒")
    print(f"批量处理加速比: {sequential_time/batch_time:.2f}x")

#### 异步 vs 同步性能对比

In [ ]:

async def async_vs_sync():
    """异步 vs 同步性能对比"""
    print("\n" + "=" * 50)
    print("异步 vs 同步性能对比")
    print("=" * 50)

    llm = Ollama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    prompts = [
        "解释机器学习",
        "解释深度学习",
        "解释神经网络"
    ]

    # 同步处理
    start_time = time.time()
    sync_responses = []
    for prompt in prompts:
        response = llm.invoke(prompt)
        sync_responses.append(response)
    sync_time = time.time() - start_time

    # 异步处理
    start_time = time.time()
    async_responses = await llm.abatch(prompts)
    async_time = time.time() - start_time

    print(f"同步处理耗时: {sync_time:.2f}秒")
    print(f"异步处理耗时: {async_time:.2f}秒")
    print(f"异步处理加速比: {sync_time/async_time:.2f}x")

#### 并发处理示例

In [ ]:

def concurrent_processing():
    """并发处理示例"""
    print("\n" + "=" * 50)
    print("并发处理示例")
    print("=" * 50)

    llm = Ollama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    prompts = [
        "什么是区块链？",
        "什么是量子计算？",
        "什么是生物计算？",
        "什么是光子计算？"
    ]

    def process_prompt(prompt: str) -> str:
        return llm.invoke(prompt)

    # 并发处理
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=4) as executor:
        concurrent_responses = list(executor.map(process_prompt, prompts))
    concurrent_time = time.time() - start_time

    # 顺序处理对比
    start_time = time.time()
    sequential_responses = [process_prompt(prompt) for prompt in prompts]
    sequential_time = time.time() - start_time

    print(f"并发处理耗时: {concurrent_time:.2f}秒")
    print(f"顺序处理耗时: {sequential_time:.2f}秒")
    print(f"并发处理加速比: {sequential_time/concurrent_time:.2f}x")

In [ ]:

def main():
    """运行所有性能优化示例"""
    caching_comparison()
    batch_vs_sequential()
    concurrent_processing()

    # 异步示例需要单独运行
    print("\n运行异步性能测试...")
    asyncio.run(async_vs_sync())

if __name__ == "__main__":
    main()


## 关键特性总结



### LLMs 核心优势
1. **简单接口**: 字符串输入输出，易于使用
2. **文本生成**: 专注于文本补全和生成任务
3. **高性能**: 支持批量、异步、缓存优化
4. **灵活性**: 可与各种提示模板组合
5. **可扩展**: 支持自定义回调和处理器

### 适用场景
- **文本生成**: 文章、代码、创意内容生成
- **文本补全**: 基于上下文的文本续写
- **批量处理**: 大量文本的批量生成
- **模板填充**: 基于模板的内容生成
- **创意写作**: 故事、诗歌等创意内容

### 性能优化建议
1. **使用缓存**: 避免重复计算相同输入
2. **批量处理**: 提高吞吐量
3. **异步调用**: 提高并发性能
4. **参数调优**: 根据需求调整温度等参数
5. **错误处理**: 实现重试和降级机制

这些示例展示了 LangChain 0.3 中 LLMs 的完整功能和最佳实践，可以根据具体需求选择合适的使用方式。




# Chat Models vs LLMs 详细对比


## 核心差异对比


In [ ]:

"""
Chat Models vs LLMs 详细对比示例
"""
from langchain_community.chat_models import ChatOllama
from langchain_ollama import OllamaLLM
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import time

def basic_interface_comparison():
    """基础接口对比"""
    print("=" * 60)
    print("1. 基础接口对比")
    print("=" * 60)

    # 初始化模型
    chat_model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    llm = OllamaLLM(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # Chat Model - 消息格式
    print("Chat Model 输入输出:")
    chat_messages = [
        SystemMessage(content="你是一个友好的助手"),
        HumanMessage(content="你好，请介绍一下自己")
    ]
    chat_response = chat_model.invoke(chat_messages)
    print(f"输入类型: {type(chat_messages)}")
    print(f"输出类型: {type(chat_response)}")
    print(f"输出内容: {chat_response.content[:100]}...")
    print(f"输出元数据: {chat_response.response_metadata}")

    print("\n" + "-" * 40)

    # LLM - 字符串格式
    print("LLM 输入输出:")
    llm_prompt = "你是一个友好的助手。用户说：你好，请介绍一下自己。请回答："
    llm_response = llm.invoke(llm_prompt)
    print(f"输入类型: {type(llm_prompt)}")
    print(f"输出类型: {type(llm_response)}")
    print(f"输出内容: {llm_response[:100]}...")

def conversation_handling():
    """对话处理对比"""
    print("\n" + "=" * 60)
    print("2. 对话处理对比")
    print("=" * 60)

    chat_model = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:3b")
    llm = Ollama(base_url="http://localhost:11434", model="qwen2.5:3b")

    # Chat Model - 自然的多轮对话
    print("Chat Model 多轮对话:")
    conversation = [
        SystemMessage(content="你是一个编程助手"),
        HumanMessage(content="我想学Python"),
        AIMessage(content="很好！Python是一门优秀的编程语言。你想从哪里开始？"),
        HumanMessage(content="从基础语法开始")
    ]

    chat_response = chat_model.invoke(conversation)
    print(f"Chat Model 回复: {chat_response.content[:150]}...")

    print("\n" + "-" * 40)

    # LLM - 需要手动构建对话上下文
    print("LLM 多轮对话:")
    llm_context = """你是一个编程助手。

对话历史:
用户: 我想学Python
助手: 很好！Python是一门优秀的编程语言。你想从哪里开始？
用户: 从基础语法开始

请回答用户的最新问题:"""

    llm_response = llm.invoke(llm_context)
    print(f"LLM 回复: {llm_response[:150]}...")

def prompt_template_comparison():
    """提示模板对比"""
    print("\n" + "=" * 60)
    print("3. 提示模板对比")
    print("=" * 60)

    chat_model = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:3b")
    llm = Ollama(base_url="http://localhost:11434", model="qwen2.5:3b")

    # Chat Model - 结构化消息模板
    print("Chat Model 提示模板:")
    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", "你是一个{role}，专门帮助用户{task}"),
        ("human", "我的问题是：{question}"),
        ("ai", "我理解你的问题，让我来帮助你。"),
        ("human", "请详细解答")
    ])

    chat_chain = chat_prompt | chat_model | StrOutputParser()
    chat_result = chat_chain.invoke({
        "role": "技术专家",
        "task": "解决编程问题",
        "question": "如何优化Python代码性能"
    })
    print(f"Chat 结果: {chat_result[:150]}...")

    print("\n" + "-" * 40)

    # LLM - 文本模板
    print("LLM 提示模板:")
    llm_prompt = PromptTemplate(
        input_variables=["role", "task", "question"],
        template="""你是一个{role}，专门帮助用户{task}。

用户问题：{question}

请详细解答："""
    )

    llm_chain = llm_prompt | llm | StrOutputParser()
    llm_result = llm_chain.invoke({
        "role": "技术专家",
        "task": "解决编程问题",
        "question": "如何优化Python代码性能"
    })
    print(f"LLM 结果: {llm_result[:150]}...")

def streaming_comparison():
    """流式处理对比"""
    print("\n" + "=" * 60)
    print("4. 流式处理对比")
    print("=" * 60)

    chat_model = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:3b")
    llm = Ollama(base_url="http://localhost:11434", model="qwen2.5:3b")

    # Chat Model 流式处理
    print("Chat Model 流式输出:")
    chat_messages = [HumanMessage(content="请写一首关于春天的短诗")]

    for chunk in chat_model.stream(chat_messages):
        print(chunk.content, end="", flush=True)
    print("\n")

    print("\n" + "-" * 40)

    # LLM 流式处理
    print("LLM 流式输出:")
    llm_prompt = "请写一首关于春天的短诗："

    for chunk in llm.stream(llm_prompt):
        print(chunk, end="", flush=True)
    print("\n")

def memory_management_comparison():
    """记忆管理对比"""
    print("\n" + "=" * 60)
    print("5. 记忆管理对比")
    print("=" * 60)

    chat_model = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:3b")
    llm = Ollama(base_url="http://localhost:11434", model="qwen2.5:3b")

    # Chat Model - 天然支持对话历史
    print("Chat Model 记忆管理:")
    chat_history = [
        SystemMessage(content="你是一个记忆力很好的助手"),
        HumanMessage(content="我的名字是张三"),
        AIMessage(content="你好张三！很高兴认识你。"),
        HumanMessage(content="我是一名程序员"),
        AIMessage(content="太好了！作为程序员，你一定很有逻辑思维。"),
        HumanMessage(content="你还记得我的名字和职业吗？")
    ]

    chat_response = chat_model.invoke(chat_history)
    print(f"Chat Model: {chat_response.content}")

    print("\n" + "-" * 40)

    # LLM - 需要手动管理上下文
    print("LLM 记忆管理:")
    llm_context = """你是一个记忆力很好的助手。

对话历史：
用户：我的名字是张三
助手：你好张三！很高兴认识你。
用户：我是一名程序员
助手：太好了！作为程序员，你一定很有逻辑思维。

当前问题：你还记得我的名字和职业吗？
请回答："""

    llm_response = llm.invoke(llm_context)
    print(f"LLM: {llm_response}")

def performance_comparison():
    """性能对比"""
    print("\n" + "=" * 60)
    print("6. 性能对比")
    print("=" * 60)

    chat_model = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:3b")
    llm = Ollama(base_url="http://localhost:11434", model="qwen2.5:3b")

    # 测试相同内容的处理时间
    test_content = "请解释什么是机器学习"

    # Chat Model 性能测试
    start_time = time.time()
    chat_response = chat_model.invoke([HumanMessage(content=test_content)])
    chat_time = time.time() - start_time

    # LLM 性能测试
    start_time = time.time()
    llm_response = llm.invoke(test_content)
    llm_time = time.time() - start_time

    print(f"Chat Model 耗时: {chat_time:.3f}秒")
    print(f"LLM 耗时: {llm_time:.3f}秒")
    print(f"性能差异: {abs(chat_time - llm_time):.3f}秒")

def use_case_examples():
    """使用场景示例"""
    print("\n" + "=" * 60)
    print("7. 使用场景示例")
    print("=" * 60)

    chat_model = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:3b")
    llm = Ollama(base_url="http://localhost:11434", model="qwen2.5:3b")

    print("Chat Model 适用场景:")
    print("1. 客服聊天机器人")
    customer_service = [
        SystemMessage(content="你是一个专业的客服代表"),
        HumanMessage(content="我的订单有问题"),
        AIMessage(content="很抱歉听到您遇到问题。请告诉我您的订单号。"),
        HumanMessage(content="订单号是12345")
    ]
    cs_response = chat_model.invoke(customer_service)
    print(f"客服回复: {cs_response.content[:100]}...")

    print("\n2. 教育对话系统")
    education = [
        SystemMessage(content="你是一个耐心的数学老师"),
        HumanMessage(content="我不理解二次方程"),
        AIMessage(content="没关系，我们一步步来学习二次方程。"),
        HumanMessage(content="什么是判别式？")
    ]
    edu_response = chat_model.invoke(education)
    print(f"老师回复: {edu_response.content[:100]}...")

    print("\n" + "-" * 40)
    print("LLM 适用场景:")

    print("1. 文本生成")
    article_prompt = "写一篇关于人工智能发展趋势的文章开头："
    article_response = llm.invoke(article_prompt)
    print(f"文章开头: {article_response[:100]}...")

    print("\n2. 代码生成")
    code_prompt = "用Python写一个快速排序算法："
    code_response = llm.invoke(code_prompt)
    print(f"代码生成: {code_response[:100]}...")

    print("\n3. 文本补全")
    completion_prompt = "人工智能的三大核心技术是机器学习、"
    completion_response = llm.invoke(completion_prompt)
    print(f"文本补全: {completion_response[:100]}...")

def main():
    """运行所有对比示例"""
    print("Chat Models vs LLMs 详细对比")
    print("=" * 80)

    basic_interface_comparison()
    conversation_handling()
    prompt_template_comparison()
    streaming_comparison()
    memory_management_comparison()
    performance_comparison()
    use_case_examples()

if __name__ == "__main__":
    main()




## 详细对比表格

| 特性维度 | Chat Models | LLMs | 说明 |
|---------|-------------|------|------|
| **输入格式** | 消息列表 (`List[BaseMessage]`) | 字符串 (`str`) | Chat Models 更结构化 |
| **输出格式** | 消息对象 (`BaseMessage`) | 字符串 (`str`) | Chat Models 包含更多元数据 |
| **角色支持** | ✅ 原生支持 (system/human/ai) | ❌ 需手动构建 | Chat Models 天然支持多角色 |
| **对话历史** | ✅ 自动管理 | ❌ 需手动维护 | Chat Models 更适合对话场景 |
| **提示模板** | `ChatPromptTemplate` | `PromptTemplate` | 不同的模板系统 |
| **流式处理** | ✅ 支持 | ✅ 支持 | 两者都支持 |
| **批量处理** | ✅ 支持 | ✅ 支持 | 两者都支持 |
| **异步支持** | ✅ 支持 | ✅ 支持 | 两者都支持 |
| **工具调用** | ✅ 原生支持 | ❌ 不支持 | Chat Models 支持 Function Calling |
| **结构化输出** | ✅ `withStructuredOutput()` | ❌ 需额外处理 | Chat Models 更强大 |
| **缓存支持** | ✅ 支持 | ✅ 支持 | 两者都支持 |
| **性能开销** | 略高 (消息解析) | 较低 (直接字符串) | LLMs 略快 |
| **使用复杂度** | 中等 | 简单 | LLMs 更简单直接 |


## 使用场景指南

In [ ]:

"""
什么时候使用 Chat Models vs LLMs 指南
"""
from langchain_community.chat_models import ChatOllama
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

def when_to_use_chat_models():
    """什么时候使用 Chat Models"""
    print("=" * 60)
    print("使用 Chat Models 的场景")
    print("=" * 60)

    chat_model = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:3b")

    print("1. 多轮对话应用")
    print("✅ 适合：聊天机器人、客服系统、教育对话")

    # 示例：客服对话
    conversation = [
        SystemMessage(content="你是专业的技术支持"),
        HumanMessage(content="我的软件崩溃了"),
        AIMessage(content="很抱歉听到这个问题。请告诉我软件版本和错误信息。"),
        HumanMessage(content="版本是2.1，显示内存不足")
    ]
    response = chat_model.invoke(conversation)
    print(f"客服回复: {response.content[:100]}...")

    print("\n2. 需要角色扮演的应用")
    print("✅ 适合：虚拟助手、角色扮演游戏、专业顾问")

    # 示例：专业顾问
    consultant = [
        SystemMessage(content="你是一位资深的投资顾问，有20年经验"),
        HumanMessage(content="我应该如何分配我的投资组合？")
    ]
    response = chat_model.invoke(consultant)
    print(f"投资顾问: {response.content[:100]}...")

    print("\n3. 需要上下文记忆的应用")
    print("✅ 适合：个人助理、学习伙伴、长期对话")

    print("\n4. 需要工具调用的应用")
    print("✅ 适合：智能代理、API集成、复杂任务执行")

    print("\n5. 需要结构化输出的应用")
    print("✅ 适合：数据提取、表单填写、API响应")

def when_to_use_llms():
    """什么时候使用 LLMs"""
    print("\n" + "=" * 60)
    print("使用 LLMs 的场景")
    print("=" * 60)

    llm = Ollama(base_url="http://localhost:11434", model="qwen2.5:3b")

    print("1. 文本生成任务")
    print("✅ 适合：文章写作、创意内容、营销文案")

    # 示例：文章生成
    article_prompt = "写一篇关于可持续发展的文章，包含引言、主体和结论："
    article = llm.invoke(article_prompt)
    print(f"文章生成: {article[:100]}...")

    print("\n2. 文本补全任务")
    print("✅ 适合：代码补全、句子续写、模板填充")

    # 示例：代码补全
    code_prompt = "def fibonacci(n):\n    if n <= 1:\n        return n\n    else:"
    code_completion = llm.invoke(code_prompt)
    print(f"代码补全: {code_completion[:100]}...")

    print("\n3. 简单的问答任务")
    print("✅ 适合：FAQ、知识查询、简单咨询")

    # 示例：知识问答
    qa_prompt = "什么是量子计算？请简单解释。"
    qa_response = llm.invoke(qa_prompt)
    print(f"问答回复: {qa_response[:100]}...")

    print("\n4. 批量文本处理")
    print("✅ 适合：文本分类、情感分析、批量翻译")

    print("\n5. 性能敏感的应用")
    print("✅ 适合：实时处理、高并发、资源受限环境")

    print("\n6. 简单的模板应用")
    print("✅ 适合：邮件模板、报告生成、格式化输出")

def decision_framework():
    """决策框架"""
    print("\n" + "=" * 60)
    print("选择决策框架")
    print("=" * 60)

    print(" 选择 Chat Models 如果你需要:")
    print("   • 多轮对话能力")
    print("   • 角色和上下文管理")
    print("   • 工具调用功能")
    print("   • 结构化输出")
    print("   • 复杂的对话逻辑")
    print("   • 消息级别的控制")

    print("\n 选择 LLMs 如果你需要:")
    print("   • 简单的文本生成")
    print("   • 最佳性能")
    print("   • 文本补全")
    print("   • 批量处理")
    print("   • 简单的接口")
    print("   • 传统的提示工程")

def practical_examples():
    """实际应用示例"""
    print("\n" + "=" * 60)
    print("实际应用示例")
    print("=" * 60)

    chat_model = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:3b")
    llm = Ollama(base_url="http://localhost:11434", model="qwen2.5:3b")

    print(" 应用类型对比:")

    applications = {
        "聊天机器人": "Chat Models",
        "客服系统": "Chat Models",
        "教育助手": "Chat Models",
        "代码助手": "Chat Models",
        "文章生成器": "LLMs",
        "翻译工具": "LLMs",
        "摘要工具": "LLMs",
        "SEO文案": "LLMs",
        "邮件模板": "LLMs",
        "数据分析报告": "LLMs",
        "智能代理": "Chat Models",
        "游戏NPC": "Chat Models",
        "内容审核": "LLMs",
        "情感分析": "LLMs"
    }

    for app, recommended in applications.items():
        emoji = "" if recommended == "Chat Models" else ""
        print(f"   {emoji} {app:<15} → {recommended}")

def migration_guide():
    """迁移指南"""
    print("\n" + "=" * 60)
    print("迁移指南")
    print("=" * 60)

    print(" 从 LLMs 迁移到 Chat Models:")
    print("   1. 将字符串提示转换为消息格式")
    print("   2. 使用 ChatPromptTemplate 替代 PromptTemplate")
    print("   3. 处理消息对象而不是字符串")
    print("   4. 利用角色系统优化提示")

    print("\n 从 Chat Models 迁移到 LLMs:")
    print("   1. 将消息列表合并为单个字符串")
    print("   2. 手动管理对话历史")
    print("   3. 使用 PromptTemplate 替代 ChatPromptTemplate")
    print("   4. 简化输出处理逻辑")

def main():
    """运行使用指南"""
    when_to_use_chat_models()
    when_to_use_llms()
    decision_framework()
    practical_examples()
    migration_guide()

if __name__ == "__main__":
    main()



## 最佳实践建议

In [ ]:

"""
Chat Models vs LLMs 最佳实践
"""
from langchain_community.chat_models import ChatOllama
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def chat_model_best_practices():
    """Chat Models 最佳实践"""
    print("=" * 60)
    print("Chat Models 最佳实践")
    print("=" * 60)

    chat_model = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:3b")

    print("1. ✅ 合理使用系统消息")
    # 好的做法
    good_messages = [
        SystemMessage(content="你是一个专业的Python编程助手，专注于提供清晰、可执行的代码解决方案"),
        HumanMessage(content="如何读取CSV文件？")
    ]

    # 不好的做法 - 系统消息过于复杂
    # bad_messages = [
    #     SystemMessage(content="你是一个超级智能的AI助手，知道所有的知识..."),
    #     HumanMessage(content="如何读取CSV文件？")
    # ]

    response = chat_model.invoke(good_messages)
    print(f"专业回复: {response.content[:100]}...")

    print("\n2. ✅ 保持对话历史简洁")
    # 只保留相关的历史消息
    relevant_history = [
        SystemMessage(content="你是编程助手"),
        HumanMessage(content="我在学Python"),
        AIMessage(content="很好！有什么具体问题吗？"),
        HumanMessage(content="如何处理异常？")  # 当前问题
    ]

    print("\n3. ✅ 使用结构化提示模板")
    structured_prompt = ChatPromptTemplate.from_messages([
        ("system", "你是{role}，专门帮助{domain}领域的问题"),
        ("human", "背景：{context}\n问题：{question}"),
        ("ai", "我理解了，让我来帮你解决这个{domain}问题。"),
        ("human", "请给出详细方案")
    ])

def llm_best_practices():
    """LLMs 最佳实践"""
    print("\n" + "=" * 60)
    print("LLMs 最佳实践")
    print("=" * 60)

    llm = Ollama(base_url="http://localhost:11434", model="qwen2.5:3b")

    print("1. ✅ 使用清晰的提示结构")
    # 好的提示结构
    good_prompt = """任务：代码生成
语言：Python
需求：实现一个计算斐波那契数列的函数
要求：
- 使用递归方法
- 包含注释
- 处理边界情况

代码："""

    response = llm.invoke(good_prompt)
    print(f"结构化提示结果: {response[:100]}...")

    print("\n2. ✅ 批量处理优化")
    # 批量处理相似任务
    batch_prompts = [
        "总结：人工智能的定义",
        "总结：机器学习的定义",
        "总结：深度学习的定义"
    ]

    batch_responses = llm.batch(batch_prompts)
    print(f"批量处理完成，处理了{len(batch_responses)}个任务")

    print("\n3. ✅ 使用模板提高复用性")
    template = PromptTemplate(
        input_variables=["topic", "style", "length"],
        template="写一篇关于{topic}的{style}风格文章，长度约{length}字："
    )

    chain = template | llm
    result = chain.invoke({
        "topic": "云计算",
        "style": "技术",
        "length": "200"
    })
    print(f"模板化结果: {result[:100]}...")

def performance_optimization():
    """性能优化建议"""
    print("\n" + "=" * 60)
    print("性能优化建议")
    print("=" * 60)

    print(" Chat Models 优化:")
    print("   • 限制对话历史长度")
    print("   • 使用消息压缩技术")
    print("   • 合理设置系统消息")
    print("   • 避免过度复杂的角色设定")

    print("\n LLMs 优化:")
    print("   • 使用批量处理")
    print("   • 启用缓存机制")
    print("   • 优化提示长度")
    print("   • 使用异步调用")

    print("\n 通用优化:")
    print("   • 合理设置温度参数")
    print("   • 使用流式处理改善用户体验")
    print("   • 实现错误重试机制")
    print("   • 监控和日志记录")

def common_pitfalls():
    """常见陷阱"""
    print("\n" + "=" * 60)
    print("常见陷阱和解决方案")
    print("=" * 60)

    print("❌ Chat Models 常见错误:")
    print("   • 过长的对话历史导致性能下降")
    print("   • 混乱的角色设定")
    print("   • 不必要的消息类型转换")
    print("   • 忽略消息元数据")

    print("\n❌ LLMs 常见错误:")
    print("   • 手动拼接复杂上下文")
    print("   • 忽略批量处理优势")
    print("   • 过度复杂的提示工程")
    print("   • 不使用模板导致重复代码")

    print("\n✅ 解决方案:")
    print("   • 定期清理对话历史")
    print("   • 使用专门的提示模板")
    print("   • 实现智能上下文管理")
    print("   • 选择合适的模型类型")

def main():
    """运行最佳实践指南"""
    chat_model_best_practices()
    llm_best_practices()
    performance_optimization()
    common_pitfalls()

if __name__ == "__main__":
    main()



## 总结建议



###  选择 Chat Models 当你需要:
1. **多轮对话**: 聊天机器人、客服系统
2. **角色扮演**: 虚拟助手、专业顾问
3. **上下文记忆**: 个人助理、学习伙伴
4. **工具集成**: 智能代理、API调用
5. **结构化交互**: 复杂的对话逻辑

###  选择 LLMs 当你需要:
1. **文本生成**: 文章、创意内容、营销文案
2. **文本补全**: 代码补全、句子续写
3. **批量处理**: 大量文本的统一处理
4. **简单问答**: FAQ、知识查询
5. **性能优先**: 高并发、低延迟场景

###  迁移建议:
- **从简单开始**: 先用 LLMs 验证概念，再升级到 Chat Models
- **渐进式迁移**: 逐步添加对话功能和角色管理
- **性能测试**: 在实际场景中对比两种方案的性能
- **用户体验**: 根据用户反馈选择最合适的交互方式

选择合适的模型类型是构建成功 AI 应用的关键第一步！
